In [1]:
import json
import os
from os.path import join, isfile
import re
import numpy as np
import pickle
import argparse
import skipthoughts
import h5py
import time

C:\Users\sbpat\AppData\Local\Continuum\anaconda3\envs\DLENV\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def save_caption_vectors(data_dir):
    img_dir = join(data_dir, "flowers/jpg")
    image_files = [f for f in os.listdir(img_dir) if "jpg" in f]
    print(image_files[300:400])
    print(len(image_files))
    image_captions = {img_file: [] for img_file in image_files}
    
    caption_dir = join(data_dir, "flowers/text_c10")
    class_dirs = []
    for i in range(1,103):
        class_dir_name = "class_%.5d"%(i)
        class_dirs.append(join(caption_dir,class_dir_name))
    
    for class_dir in class_dirs:
        caption_files = [f for f in os.listdir(class_dir) if "txt" in f]
        for cap_file in caption_files:
            with open(join(class_dir,cap_file)) as f:
                captions = f.read().split("\n")
            img_file = cap_file[0:11] + ".jpg"
            #5 captions per image
            image_captions[img_file] += [cap for cap in captions if len(cap)>0][0:5]
    
    print(len(captions))
    
    model = skipthoughts.load_model()
    encoded_captions = {}
    for i,img in enumerate(image_captions):
        st = time.time()
        encoded_captions[img] = skipthoughts.encode(model,image_captions[img])
        print(i,len(image_captions),img)
        print("Seconds", time.time() - st)
        
    h = h5py.File(join(data_dir,"flower_tv.hdf5"))
    for key in encoded_captions:
        h.create_dataset(key, data = encoded_captions[key])
    h.close()

In [ ]:
save_caption_vectors('Data')

In [3]:
import tensorflow as tf
import ops

class GAN:
    def __init__(self,options):
        self.options = options
        self.g_bn0 = ops.batch_norm(name = "g_bn0")
        self.g_bn1 = ops.batch_norm(name = "g_bn1")
        self.g_bn2 = ops.batch_norm(name = "g_bn2")
        self.g_bn3 = ops.batch_norm(name = "g_bn3")
        
        self.d_bn1 = ops.batch_norm(name = "d_bn1")
        self.d_bn2 = ops.batch_norm(name = "d_bn2")
        self.d_bn3 = ops.batch_norm(name = "d_bn3")
        self.d_bn4 = ops.batch_norm(name = "d_bn4")
    #DC-GAN generator    
    def generator(self,t_z,t_text_embedding):
        with tf.variable_scope(tf.get_variable_scope(),reuse = tf.AUTO_REUSE):
            s = self.options["image_size"]

            s2, s4, s8, s16 = int(s/2), int(s/4), int(s/8), int(s/16)

            reduced_text_embedding = ops.lrelu(ops.linear(t_text_embedding,self.options["t_dim"],"g_embedding"))
            z_concat = tf.concat( [t_z, reduced_text_embedding],1)
            z_ = ops.linear(z_concat, self.options["gf_dim"]*8*s16*s16, "g_h0_lin")
            h0 = tf.reshape(z_, [-1,s16,s16, self.options["gf_dim"]*8])
            h0 = tf.nn.relu(self.g_bn0(h0))

            h1 = ops.deconv2d(h0, [self.options["batch_size"], s8, s8, self.options["gf_dim"]*4],name = "g_h1")
            h1 = tf.nn.relu(self.g_bn1(h1))

            h2 = ops.deconv2d(h1, [self.options["batch_size"], s4, s4, self.options["gf_dim"]*2],name = "g_h2")
            h2 = tf.nn.relu(self.g_bn2(h2))

            h3 = ops.deconv2d(h2, [self.options["batch_size"], s2, s2, self.options["gf_dim"]*1],name = "g_h3")
            h3 = tf.nn.relu(self.g_bn3(h3))

            h4 = ops.deconv2d(h3, [self.options["batch_size"], s, s, 3], name = "g_h4")

        return (tf.tanh(h4)/2. + 0.5)
    
    #DC-GAN discriminator
    def discriminator(self,image,t_text_embedding,reuse = True):
        #64 -> 32
        with tf.variable_scope(tf.get_variable_scope(), reuse = tf.AUTO_REUSE):
            h0 = ops.lrelu(ops.conv2d(image,self.options["df_dim"], name = "d_h0_conv"))
            #32 -> 16
            h1 = ops.lrelu(self.d_bn1(ops.conv2d(h0,self.options["df_dim"]*2, name = "d_h1_conv")))
            #16 -> 8
            h2 = ops.lrelu(self.d_bn2(ops.conv2d(h1,self.options["df_dim"]*4, name = "d_h2_conv")))
            #8 -> 4
            h3 = ops.lrelu(self.d_bn3(ops.conv2d(h2,self.options["df_dim"]*8, name = "d_h3_conv")))

            reduced_text_embeddings = ops.lrelu(ops.linear(t_text_embedding,self.options["t_dim"],"d_embedding"))
            reduced_text_embeddings = tf.expand_dims(reduced_text_embeddings, 1)
            reduced_text_embeddings = tf.expand_dims(reduced_text_embeddings, 2)
            tiled_embeddings = tf.tile(reduced_text_embeddings,[1,4,4,1], name = "tiled_embeddings")
            h3_concat = tf.concat( [h3,tiled_embeddings],3, name = "h3_concat")
            h3_new = ops.lrelu(self.d_bn4(ops.conv2d(h3_concat,self.options["df_dim"]*8,1,1,1,1, name = "d_h3_conv_new")))

            h4 = ops.linear(tf.reshape(h3_new, [ self.options["batch_size"],-1]), 1, "d_h3_lin")
        
        return tf.nn.sigmoid(h4), h4
        
    def build_model(self):
        img_size = self.options["image_size"]
        t_real_image = tf.placeholder("float32",[self.options["batch_size"],img_size,img_size,3],name="real_image")
        t_wrong_image = tf.placeholder("float32",[self.options["batch_size"],img_size,img_size,3],name="wrong_image")
        t_real_caption = tf.placeholder("float32",[self.options["batch_size"],self.options["caption_vector_length"]],name = "real_caption_input")
        t_z = tf.placeholder("float32",[self.options["batch_size"],self.options["z_dim"]])
        
        fake_image = self.generator(t_z,t_real_caption)
        
        disc_real_image, disc_real_image_logits = self.discriminator(t_real_image,t_real_caption)
        disc_wrong_image, disc_wrong_image_logits = self.discriminator(t_wrong_image,t_real_caption, reuse = True)
        disc_fake_image, disc_fake_image_logits = self.discriminator(fake_image,t_real_caption,reuse = True)
        
        g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_fake_image_logits,labels= tf.ones_like(disc_fake_image)))
        
        d_loss1 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_real_image_logits,labels=tf.ones_like(disc_real_image)))
        d_loss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_wrong_image_logits,labels=tf.zeros_like(disc_wrong_image)))
        d_loss3 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_fake_image_logits,labels=tf.zeros_like(disc_fake_image)))
        
        d_loss = d_loss1 + d_loss2 + d_loss3
        t_vars = tf.trainable_variables()
        d_vars = [var for var in t_vars if "d_" in var.name]
        g_vars = [var for var in t_vars if "g_" in var.name]
        
        input_tensors = {
            "t_real_image": t_real_image,
            "t_wrong_image": t_wrong_image,
            "t_real_caption": t_real_caption,
            "t_z": t_z
        }
        variables = {
            "d_vars": d_vars,
            "g_vars": g_vars
        }
        loss = {
            "g_loss":g_loss,
            "d_loss": d_loss
        }
        outputs = {"generator": fake_image}
        checks = {
            "d_loss1": d_loss1,
            "d_loss2": d_loss2,
            "d_loss3": d_loss3,
            "disc_real_image_logits": disc_real_image_logits,
            "disc_wrong_image_logits": disc_wrong_image,
            "disc_fake_image_logits": disc_fake_image_logits
        }
        return input_tensors, variables,loss,outputs,checks
    
    def build_generator(self):
        img_size = self.options["image_size"]
        t_real_caption = tf.placeholder("float32",[self.options["batch_size"],self.options["caption_vector_length"]],name = "real_caption_input")
        t_z = tf.placeholder("float32",[self.options["batch_size"],self.options["z_dim"]])
        fake_image = self.sampler(t_z,t_real_caption)
        
        input_tensors = {"t_real_caption":t_real_caption,"t_z": t_z}
        outputs = {"generator":fake_image}
        return input_tensors,outputs
    
    def sampler(self,t_z, t_text_embedding):
        with tf.variable_scope(tf.get_variable_scope(),reuse = tf.AUTO_REUSE):
            s = self.options["image_size"]
            s2, s4, s8, s16 = int(s/2), int(s/4), int(s/8), int(s/16)

            reduced_text_embedding = ops.lrelu(ops.linear(t_text_embedding,self.options["t_dim"],"g_embedding"))
            z_concat = tf.concat([t_z, reduced_text_embedding],1)
            z_ = ops.linear(z_concat,self.options["gf_dim"]*8*s16*s16,"g_h0_lin")

            h0 = tf.reshape(z_,[-1,s16,s16,self.options["gf_dim"]*8])
            h0 = tf.nn.relu(self.g_bn0(h0,train = False))

            h1 = ops.deconv2d(h0, [self.options["batch_size"],s8,s8,self.options["gf_dim"]*4],name = "g_h1")
            h1 = tf.nn.relu(self.g_bn1(h1,train = False))

            h2 = ops.deconv2d(h1, [self.options["batch_size"],s4,s4,self.options["gf_dim"]*2],name = "g_h2")
            h2 = tf.nn.relu(self.g_bn2(h2,train = False))

            h3 = ops.deconv2d(h2, [self.options["batch_size"],s2,s2,self.options["gf_dim"]*1],name = "g_h3")
            h3 = tf.nn.relu(self.g_bn3(h3,train = False))

            h4 = ops.deconv2d(h3, [self.options["batch_size"],s,s,3], name = "g_h4")
        
        return (tf.tanh(h4)/2. + 0.5)

In [4]:
import scipy.misc
import image_processing
import random
import shutil
import imageio
from skimage import img_as_ubyte

In [5]:
def main():
    z_dim = 100
    t_dim = 256
    batch_size = 256
    image_size = 64
    gf_dim = 64
    df_dim = 64
    gfc_dim = 1024
    caption_vector_length = 2400
    data_dir = "Data"
    learning_rate = 0.0003
    beta1 = 0.5
    epochs = 200
    save_every = 20
    resume_model = True
    data_set = "flowers"
    model_options = {
        "z_dim" : z_dim,
        "t_dim" : t_dim,
        "batch_size" : batch_size,
        "image_size" : image_size,
        "gf_dim" : gf_dim,
        "df_dim" : df_dim,
        "gfc_dim" : gfc_dim,
        "caption_vector_length" : caption_vector_length
    }
    gan = GAN(model_options)
    input_tensors,variables,loss,outputs,checks = gan.build_model()
    d_optim = tf.train.AdamOptimizer(learning_rate,beta1 = beta1).minimize(loss["d_loss"],var_list = variables["d_vars"])
    g_optim = tf.train.AdamOptimizer(learning_rate,beta1 = beta1).minimize(loss["g_loss"],var_list = variables["g_vars"])
    
    sess = tf.InteractiveSession()
    tf.initialize_all_variables().run()
    
    saver = tf.train.Saver()
    if resume_model:
        saver = tf.train.import_meta_graph('Data/Models/latest_mdoel_flowers_temp.ckpt.index')
        saver.restore(sess, "Data/Models/latest_mdoel_flowers_temp.ckpt")
        
    loaded_data = load_training_data(data_dir, data_set)
    
    for i in range(epochs):
        batch_no = 0
        while batch_no*batch_size < loaded_data["data_length"]:
            real_images,wrong_images,caption_vectors,z_noise,image_files = get_training_batch(batch_no,batch_size,
                image_size,z_dim,caption_vector_length,"train",data_dir,data_set, loaded_data)
            
            #Update Discriminator
            check_ts = [checks["d_loss1"], checks["d_loss2"], checks["d_loss3"]]
            _, d_loss, gen,d1,d2,d3 = sess.run([d_optim, loss["d_loss"], outputs["generator"]]+check_ts,
                                              feed_dict={
                                                  input_tensors["t_real_image"]: real_images,
                                                  input_tensors["t_wrong_image"]: wrong_images,
                                                  input_tensors["t_real_caption"]: caption_vectors,
                                                  input_tensors["t_z"]: z_noise,
                                              })
            
            print("d1",d1)
            print("d2",d2)
            print("d3",d3)
            print("D",d_loss)
            #Update Generator
            _, g_loss, gen = sess.run([g_optim, loss["g_loss"], outputs["generator"]],
                feed_dict = {
                    input_tensors["t_real_image"] : real_images,
                    input_tensors["t_wrong_image"] : wrong_images,
                    input_tensors["t_real_caption"] : caption_vectors,
                    input_tensors["t_z"] : z_noise,
                })
            # GEN UPDATE TWICE, to make sure d_loss does not go to 0
            _, g_loss, gen = sess.run([g_optim, loss["g_loss"], outputs["generator"]],
                feed_dict = {
                    input_tensors["t_real_image"] : real_images,
                    input_tensors["t_wrong_image"] : wrong_images,
                    input_tensors["t_real_caption"] : caption_vectors,
                    input_tensors["t_z"] : z_noise,
                })
            print("LOSSES",d_loss,g_loss,batch_no,i,len(loaded_data["image_list"])/batch_size)
            batch_no +=1
            if (batch_no%save_every) == 0:
                print("Saving Images, Model")
                save_for_vis(data_dir,real_images,gen,image_files)
                save_path = saver.save(sess,"Data/Models/latest_mdoel_{}_temp.ckpt".format(data_set))
        if i%5==0:
            save_path = saver.save(sess,"Data/Models/model_after_{}_epoch_{}.ckpt".format(data_set,i))
            
def load_training_data(data_dir, data_set):
    if data_set == "flowers":
        h = h5py.File(join(data_dir, "flower_tv.hdf5"))
        flower_captions = {}
        for ds in h.items():
            flower_captions[ds[0]] = np.array(ds[1])
        image_list = [key for key in flower_captions]
        image_list.sort()

        img_75 = int(len(image_list)*0.75)
        training_image_list = image_list[0:img_75]
        random.shuffle(training_image_list)
        
        return {
            "image_list" : training_image_list,
            "captions" : flower_captions,
            "data_length" : len(training_image_list)
        }
    
    else:
        with open(join(data_dir, "meta_train.pkl")) as f:
            meta_data = pickle.load(f)
        # No preloading for MS-COCO
        return meta_data
            
def save_for_vis(data_dir, real_images, generated_images, image_files):
    #create a set of 10 random numbers to save 10 random images in samples directory
    a= set()
    while len(a)<=10:
        a.update(random.sample(range(*sorted([0,real_images.shape[0]])),10))
    
    for i in list(a)[:10]:
        real_image_255 = np.zeros( (64,64,3), dtype=np.uint8)
        real_images_255 = img_as_ubyte((real_images[i,:,:,:]))
        imageio.imwrite( join(data_dir, "samples/{}_{}.jpg".format(i, image_files[i].split("/")[-1] )) , real_images_255)

        fake_image_255 = np.zeros( (64,64,3), dtype=np.uint8)
        fake_images_255 = img_as_ubyte((generated_images[i,:,:,:]))
        imageio.imwrite(join(data_dir, "samples/fake_image_{}.jpg".format(i)), fake_images_255)
        
def get_training_batch(batch_no, batch_size, image_size, z_dim, 
    caption_vector_length, split, data_dir, data_set, loaded_data = None):
    if data_set == "mscoco":
        with h5py.File( join(data_dir, "tvs/"+split + "_tvs_" + str(batch_no))) as hf:
            caption_vectors = np.array(hf.get("tv"))
            caption_vectors = caption_vectors[:,0:caption_vector_length]
        with h5py.File( join(data_dir, "tvs/"+split + "_tv_image_id_" + str(batch_no))) as hf:
            image_ids = np.array(hf.get("tv"))

        real_images = np.zeros((batch_size, 64, 64, 3))
        wrong_images = np.zeros((batch_size, 64, 64, 3))
        
        image_files = []
        for idx, image_id in enumerate(image_ids):
            image_file = join(data_dir, "%s2014/COCO_%s2014_%.12d.jpg"%(split, split, image_id) )
            image_array = image_processing.load_image_array(image_file, image_size)
            real_images[idx,:,:,:] = image_array
            image_files.append(image_file)
        
        # TODO>> As of Now, wrong images are just shuffled real images.
        first_image = real_images[0,:,:,:]
        for i in range(0, batch_size):
            if i < batch_size - 1:
                wrong_images[i,:,:,:] = real_images[i+1,:,:,:]
            else:
                wrong_images[i,:,:,:] = first_image
        z_noise = np.random.uniform(-1, 1, [batch_size, z_dim])


        return real_images, wrong_images, caption_vectors, z_noise, image_files

    if data_set == "flowers":
        real_images = np.zeros((batch_size, 64, 64, 3))
        wrong_images = np.zeros((batch_size, 64, 64, 3))
        captions = np.zeros((batch_size, caption_vector_length))

        cnt = 0
        image_files = []
        for i in range(batch_no * batch_size, batch_no * batch_size + batch_size):
            idx = i % len(loaded_data["image_list"])
            image_file =  join(data_dir, "flowers/jpg/"+loaded_data["image_list"][idx])
            image_array = image_processing.load_image_array(image_file, image_size)
            real_images[cnt,:,:,:] = image_array
            
            # Improve this selection of wrong image
            wrong_image_id = random.randint(0,len(loaded_data["image_list"])-1)
            wrong_image_file =  join(data_dir, "flowers/jpg/"+loaded_data["image_list"][wrong_image_id])
            wrong_image_array = image_processing.load_image_array(wrong_image_file, image_size)
            wrong_images[cnt, :,:,:] = wrong_image_array

            random_caption = random.randint(0,4)
            captions[cnt,:] = loaded_data["captions"][ loaded_data["image_list"][idx] ][ random_caption ][0:caption_vector_length]
            image_files.append( image_file )
            cnt += 1

        z_noise = np.random.uniform(-1, 1, [batch_size, z_dim])
        return real_images, wrong_images, captions, z_noise, image_files

In [ ]:
main()

In [6]:
def generate_thought_vectors():
    caption_file = "Data/sample_captions.txt"
    data_dir = "Data"
    with open(caption_file) as f:
        captions = f.read().split("\n")
    captions = [cap for cap in captions if len(cap)>0]
    print(captions)
    model = skipthoughts.load_model()
    caption_vectors = skipthoughts.encode(model,captions)
    if os.path.isfile(join(data_dir, "sample_caption_vectors.hdf5")):
        os.remove(join(data_dir,"sample_caption_vectors.hdf5"))
    h = h5py.File(join(data_dir,"sample_caption_vectors.hdf5"))
    h.create_dataset("vectors", data = caption_vectors)
    h.close()

In [ ]:
generate_thought_vectors()

In [7]:
import model
import ops
import cv2
import PIL

In [8]:
def generate_images():
    z_dim = 100
    t_dim = 256
    image_size = 64
    gf_dim = 64
    df_dim = 64
    gfc_dim = 1024
    caption_vector_length = 2400
    data_dir = "Data"
    model_path = "Data/Models/latest_model_flowers_temp.ckpt"
    n_images = 400
    caption_thought_vectors = "Data/sample_caption_vectors.hdf5"
    model_options = {
        "z_dim" : z_dim,
        "t_dim" : t_dim,
        "batch_size" : n_images,
        "image_size" : image_size,
        "gf_dim" : gf_dim,
        "df_dim" : df_dim,
        "gfc_dim" : gfc_dim,
        "caption_vector_length" : caption_vector_length
    }
    gan = GAN(model_options)
    _,_,_,_,_ = gan.build_model()
    sess = tf.InteractiveSession()
    
#     saver = tf.train.Saver()
#     saver.restore(sess,model_path)
    saver = tf.train.import_meta_graph('Data/Models/latest_mdoel_flowers_temp.ckpt.index')
    saver.restore(sess, "Data/Models/latest_mdoel_flowers_temp.ckpt")
    
    input_tensors, outputs = gan.build_generator()
    
    h = h5py.File(caption_thought_vectors)
    caption_vectors = np.array(h["vectors"])
    caption_image_dic = {}
    for cn, caption_vectors in enumerate(caption_vectors):
        caption_images = []
        z_noise = np.random.uniform(-1,1,[n_images, z_dim])
        caption = [caption_vectors[0:caption_vector_length]]*n_images
        [gen_image] = sess.run([outputs["generator"]],
            feed_dict = {
                input_tensors["t_real_caption"]: caption,
                input_tensors["t_z"]: z_noise,
            })
        caption_images = [gen_image[i,:,:,:] for i in range(0,n_images)]
        caption_image_dic[cn] = caption_images
        print("Generated", cn)
    for f in os.listdir(join(data_dir, "val_samples")):
        if os.path.isfile(f):
            os.unlink(join(data_dir,"val_samples/"+f))
    
    for cn in range(0, len(caption_vectors)-1):
        caption_images = []
        for i, im in enumerate( caption_image_dic[ cn ] ):
            # im_name = "caption_{}_{}.jpg".format(cn, i)
            # scipy.misc.imsave( join(args.data_dir, 'val_samples/{}'.format(im_name)) , im)
            caption_images.append( im )
            scipy.misc.imsave( join(data_dir, 'val_samples/combined_image_%d_%04d.jpg' %(cn,i)) , im)
        caption_images = np.array(caption_images)
        caption_images = (caption_images*255).astype(np.uint8)
        make_video(caption_images)
        
            
#     for cn in range(0, len(caption_vectors)):
#         caption_images = []
#         for i, im in enumerate( caption_image_dic[ cn ] ):
#             caption_images.append( im )
            
#         make_video(caption_images)
#         for  i in range(len(caption_images)):
#             imageio.imwrite(join(data_dir, "val_samples/{}.jpg".format(i)), np.uint8(caption_images[i]))

In [10]:
generate_images()

INFO:tensorflow:Restoring parameters from Data/Models/latest_mdoel_flowers_temp.ckpt


C:\Users\sbpat\AppData\Local\Continuum\anaconda3\envs\DLENV\lib\site-packages\tensorflow\python\framework\meta_graph.py:637: RuntimeWarning: Unexpected end-group tag: Not all data was converted
  meta_graph_def.ParseFromString(file_content)


Generated 0
Generated 1
Generated 2


C:\Users\sbpat\AppData\Local\Continuum\anaconda3\envs\DLENV\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


[[[[1.55701637e-01 1.95370972e-01 1.17713958e-01]
   [9.70563293e-02 1.17971867e-01 8.81196260e-02]
   [1.08201563e-01 8.74434114e-02 1.10710263e-01]
   ...
   [2.15970486e-01 2.30552971e-01 3.01989853e-01]
   [2.18476713e-01 2.22276211e-01 3.13048124e-01]
   [2.45469451e-01 2.82682091e-01 3.06617141e-01]]

  [[8.30275118e-02 1.22683585e-01 6.46986663e-02]
   [1.11763656e-01 9.35676694e-02 7.38447011e-02]
   [1.56059355e-01 1.13070339e-01 1.18269622e-01]
   ...
   [2.25871563e-01 2.27404714e-01 2.71698892e-01]
   [3.20263445e-01 3.30670834e-01 3.71549308e-01]
   [3.08630288e-01 3.27499062e-01 3.74122441e-01]]

  [[6.35487437e-02 1.06324434e-01 5.85293174e-02]
   [1.60197467e-01 1.26789629e-01 1.35137618e-01]
   [1.18559748e-01 1.12850845e-01 1.34008139e-01]
   ...
   [2.75586784e-01 2.50427544e-01 2.13617921e-01]
   [2.53101289e-01 3.29911172e-01 2.76510537e-01]
   [2.86723167e-01 3.18481147e-01 3.07479739e-01]]

  ...

  [[8.49150419e-02 1.22905910e-01 2.03258514e-01]
   [1.22577429e-

[[[[0.3585397  0.38427362 0.1873034 ]
   [0.34275386 0.33972532 0.19669896]
   [0.3346174  0.36325538 0.23110712]
   ...
   [0.18620217 0.22153682 0.20052606]
   [0.17090386 0.2060996  0.15411419]
   [0.23523706 0.3077317  0.22811192]]

  [[0.26265436 0.3734489  0.1180574 ]
   [0.44007817 0.45144156 0.22402799]
   [0.3605659  0.43314347 0.22910559]
   ...
   [0.26853243 0.3014543  0.20022264]
   [0.32813105 0.38888845 0.23588678]
   [0.31875855 0.3365627  0.24535578]]

  [[0.22348082 0.35225886 0.10507321]
   [0.52097887 0.5338098  0.25171256]
   [0.33868557 0.4970353  0.26782006]
   ...
   [0.30584162 0.27914783 0.20185652]
   [0.24435437 0.32073635 0.19436553]
   [0.32704377 0.34227562 0.23630148]]

  ...

  [[0.39334625 0.44634235 0.50365835]
   [0.36461502 0.52076864 0.4286363 ]
   [0.37982216 0.47177076 0.37933496]
   ...
   [0.55298513 0.49980122 0.37816292]
   [0.41513255 0.48704597 0.3941809 ]
   [0.36045253 0.3570283  0.28015983]]

  [[0.40364838 0.49321097 0.48455137]
   [0.3

[[[[0.30134475 0.27437586 0.13375095]
   [0.14898437 0.1559414  0.08104396]
   [0.12324589 0.10978347 0.07841235]
   ...
   [0.04398251 0.0809558  0.00937006]
   [0.07145169 0.08701584 0.0133065 ]
   [0.05739182 0.09264696 0.01898858]]

  [[0.23163056 0.2589221  0.0897609 ]
   [0.09508914 0.10520938 0.04528236]
   [0.14721376 0.13367385 0.08273959]
   ...
   [0.07972187 0.11776841 0.00665423]
   [0.10405567 0.15992436 0.00928143]
   [0.05880016 0.07231033 0.00718322]]

  [[0.10966811 0.19362104 0.0561409 ]
   [0.08542791 0.13087481 0.0439463 ]
   [0.13138878 0.17682004 0.07235396]
   ...
   [0.10497922 0.15200603 0.01029798]
   [0.0817073  0.1288957  0.00625849]
   [0.06503087 0.11117631 0.0075548 ]]

  ...

  [[0.22345537 0.289669   0.19465852]
   [0.21656585 0.25572395 0.26024753]
   [0.21826667 0.2890537  0.34289557]
   ...
   [0.3208584  0.25239962 0.14935136]
   [0.28672534 0.41523528 0.20184931]
   [0.32430905 0.44165045 0.24752766]]

  [[0.16409251 0.22186649 0.11697018]
   [0.2

KeyError: 3

In [9]:
def make_video(images, fps=25, size=None,
               is_color=True, format="XVID"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
 
    The function relies on http://opencv-python-tutroals.readthedocs.org/en/latest/.
    By default, the video will have the size of the first image.
    It will resize every image to this size before adding them to the video.
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = cv2.VideoWriter('output.avi', fourcc, 25.0, (64,64))
    
    for image in images:
        img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        vid.write(img)
    vid.release()
    cv2.destroyAllWindows()
    return

In [ ]:
from tensorflow.python import pywrap_tensorflow
import os

checkpoint_path = os.path.join('', 'Data/Models/latest_model_flowers_temp.ckpt')
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
var_to_shape_map = reader.get_variable_to_shape_map()

for key in var_to_shape_map:
    print("tensor_name: ", key)